In [1]:
from qiime2 import Artifact
import qiime2 as q2
import biom

In [2]:
#import metadata
mf = q2.Metadata.load('14383_20220203-201511.txt').to_dataframe()

In [3]:
# #create a new storage temp condition for samples that were put in -20C after 1 week
# mf['sample_storage_temp_treatment_week_added'] = [j + ' after one week' 
#                                                   if '20C.A' in i else j 
#                                                   for i, j in zip(mf.index,mf.sample_storage_temp_treatment)]

In [4]:
# mf['fresh_or_not'] = ['yes' if i == 'fresh' else 'no' 
#                       for i in mf.duration_of_storage]

In [5]:
#import table into qiime2 and filter out blanks, controls, and mistakes

#import table 
bt = biom.load_table('feature-table.biom')

bt_id_map =  {i: i[7:] for i in bt.ids()}
bt.update_ids(bt_id_map)

#identify the metadata data to keep 
keep = [ind for ind in mf.index if 'urine' in ind]

#indentify the biom table data to keep (match to metadata)
keep2 = list(set(keep) & set(bt.ids()))

#filter metadata
mf_filtered = mf.loc[keep2, : ]

#filter biom table
bt_filter = bt.filter(mf_filtered.index, axis='sample')

#some features may have been removed that were unique to the samples removed
#now there may be a lot of 0's in our table, remove them and match tables
keep_feat = bt_filter.ids('observation')[bt_filter.sum(axis='observation') > 0]
bt_filter = bt_filter.filter(keep_feat, axis='observation')

#turn tables to q2 artifacts
q2mf_filtered = q2.Metadata(mf_filtered)
q2mf_filtered.save('filtered-metadata.qza')
q2bt = q2.Artifact.import_data('FeatureTable[Frequency]', bt_filter)
q2bt.save('filtered-table.qza')

'filtered-table.qza'

In [6]:
bt_filter

1989 x 180 <class 'biom.table.Table'> with 8227 nonzero entries (2% dense)